### 策略逻辑
* 分钟级别回测
* 上轨：昨日收盘价 * (1 + 0.03), 下轨：昨日收盘价 * (1 - 0.03)
* 下单逻辑：当分钟close突破上轨时卖出，当分钟close突破下轨时买入，收盘时根据当天买卖情况平仓


In [3]:
# 本代码由可视化策略环境自动生成 2021年3月30日11:20
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：初始化函数，只执行一次
def m19_initialize_bigquant_run(context):
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    # 最低底仓
    context.min_amount = 500
    # 每次交易股数
    context.per_order_amount = 1000
    # 建仓标志
    context.first_order_flag = True
    # 今日已发单标志
    context.today_order_flag = False
    # 已发买单标志
    context.buy_flag = False
    # 已发卖单标志
    context.sell_flag = False
    # 通道上轨、下轨
    context.up_track, context.down_track = 0.0, 0.0
    
# 回测引擎：每日数据处理函数，每天执行一次
def m19_handle_data_bigquant_run(context, data):
    if context.first_order_flag:
        print("建立底仓")
        # 第一天买入, 设置底仓
        context.order(context.symbol("603363.SHA"), 2000)
        context.last_order_price = data.current(context.symbol("603363.SHA"), "close")
        context.first_order_flag = False
        return
    if context.trading_day_index < 1:
        # 首日除了建仓，不进行其他发单操作
        return
    
    # 当天第一根分钟行情到来时,打开交易标志位
    if data.current_dt.strftime("%H%M%S") == "093100":
        # 计算今天交易的上轨和下轨
        sid = context.symbol("603363.SHA")
        pre_close = data.history(sid, fields="close", bar_count=1, frequency="1d").values[0]
        context.up_track, context.down_track = pre_close * (1+0.03), pre_close * (1-0.03)
        
        context.today_order_flag = True
        context.buy_flag = False
        context.sell_flag = False
    # 产生下单信号
    sid = context.symbol("603363.SHA")
    close = data.current(sid, "close")
    sell_signal = close > context.up_track
    buy_signal = close < context.down_track
    # 买入信号
    if buy_signal and context.today_order_flag:
        context.order(sid, context.per_order_amount)
        context.today_order_flag = False
        context.buy_flag = True
    # 卖出信号
    if sell_signal and context.today_order_flag:
        if context.portfolio.positions[sid].amount > context.min_amount:
            context.order(sid, (-1) * context.per_order_amount)
            context.today_order_flag = False
            context.sell_flag = True
    
    # 收盘时卖出或买入平仓
    if data.current_dt.strftime("%H%M%S") == "145900" and context.buy_flag:
        context.order(sid, (-1) * context.per_order_amount)
    if data.current_dt.strftime("%H%M%S") == "145900" and context.sell_flag:
        context.order(sid, context.per_order_amount)
# 回测引擎：准备数据，只执行一次
def m19_prepare_bigquant_run(context):
    pass


m1 = M.instruments.v2(
    start_date='2019-01-03',
    end_date='2021-03-27',
    market='CN_STOCK_A',
    instrument_list='603363.SHA',
    max_count=0
)

m19 = M.trade.v4(
    instruments=m1.data,
    start_date='',
    end_date='',
    initialize=m19_initialize_bigquant_run,
    handle_data=m19_handle_data_bigquant_run,
    prepare=m19_prepare_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=100000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='minute',
    price_type='真实价格',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='000300.SHA'
)


[2021-03-30 11:17:22.430193] INFO: moduleinvoker: instruments.v2 开始运行..

[2021-03-30 11:17:22.436614] INFO: moduleinvoker: 命中缓存

[2021-03-30 11:17:22.437845] INFO: moduleinvoker: instruments.v2 运行完成[0.007661s].

[2021-03-30 11:17:22.507773] INFO: moduleinvoker: backtest.v8 开始运行..

[2021-03-30 11:17:22.512458] INFO: backtest: biglearning backtest:V8.5.0

...

[2021-03-30 11:17:55.911280] INFO: Performance: Simulated 541 trading days out of 541.

[2021-03-30 11:17:55.912843] INFO: Performance: first open: 2019-01-03 09:30:00+00:00

[2021-03-30 11:17:55.913948] INFO: Performance: last close: 2021-03-26 15:00:00+00:00

[2021-03-30 11:17:58.203485] INFO: moduleinvoker: backtest.v8 运行完成[35.695699s].

[2021-03-30 11:17:58.204998] INFO: moduleinvoker: trade.v4 运行完成[35.764113s].